# Setup

In [2]:
# install dependencies
!pipenv install langchain
!pipenv install sentence_transformers
!pipenv install chromadb
!pipenv install unstructured

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain...
Resolving langchain...
✔ Installation Succeeded
⠋ Installing langchain...
Pipfile.lock (21ba60) out of date, updating to (d3d6e2)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠇ Locking...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠦ Locking...
Updated Pipfile.lock (710967f062910ffce98e65863f099e0664f6382fb08b7aa2a791a6207dd3d6e2)!
Installing dependencies from Pipfile.lock (d3d6e2)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a co

In [3]:
# load dependencies
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [4]:
# set params
DATA_PATH = "data/html"
CHROMA_PATH = "chroma_db"
EMBED_MODEL = "all-MiniLM-L6-v2" # Chroma defaults to "sentence-transformers/all-MiniLM-L6-v2"

In [5]:
# load docs
def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(DATA_PATH)
len(documents)

[nltk_data] Downloading package punkt to /Users/steve/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/steve/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


3487

In [6]:
documents[0]

Document(page_content='46.708 Warranties of data.\n\nWarranties of data shall be developed and used in accordance with agency regulations.\n\nSubpart 46.7 - Warranties', metadata={'source': 'data/html/46.708.html'})

In [7]:
# define text embedding model
embedding_func = SentenceTransformerEmbeddings(model_name=EMBED_MODEL) # "BAAI/bge-small-en-v1.5"

# See https://huggingface.co/spaces/mteb/leaderboard

/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.40MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 984kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 14.0MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 296kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 445kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:03<00:00, 29.8MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 111kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 388kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.69MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00,

In [8]:
# initialize Chroma db and save to disk
db = Chroma.from_documents(
    documents=documents, embedding=embedding_func, persist_directory=CHROMA_PATH
    )

db.persist()

# print message
print(f"Saved {len(documents)} chunks to {CHROMA_PATH}.")

In [10]:
# query vector db
query = "What is the purpose of the Federal Acquisition Regulations System?"
matching_docs = db.similarity_search(query)

matching_docs

[Document(page_content='1.101 Purpose.\n\nThe Federal Acquisition Regulations System is established for the codification and publication of uniform policies and procedures for acquisition by all executive agencies. The Federal Acquisition Regulations System consists of the Federal Acquisition Regulation (FAR), which is the primary document, and agency acquisition regulations that implement or supplement the FAR. The FAR System does not include internal agency guidance of the type described in 1.301(a)(2).\n\nSubpart 1.1 - Purpose, Authority, Issuance', metadata={'source': 'data/html/1.101.html'}),
 Document(page_content='1.000 Scope of part.\n\nThis part sets forth basic policies and general information about the Federal Acquisition Regulations System including purpose, authority, applicability, issuance, arrangement, numbering, dissemination, implementation, supplementation, maintenance, administration, and deviation. subparts\xa0 1.2,1.3, and 1.4 prescribe administrative procedures f